In [213]:
# librerías
import torch
from torch import nn
from torch.nn import Module
from torch.nn import Linear
from torch.nn import Sequential
from collections import OrderedDict
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.nn import functional as F
import numpy

# seed fija para experimentos
seed = 7
numpy.random.seed(seed)

In [138]:
inputs.shape,Y.shape


(torch.Size([768, 8]), torch.Size([768, 1]))

In [214]:
#cargar los datos
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# separamos en variables de entrada (X) y salidas (Y) 
inputs = torch.from_numpy(dataset[:,0:8]).float()
X = DataLoader(inputs, batch_size=10, shuffle=True)
Y = torch.reshape(torch.from_numpy(dataset[:,8]).float(), (-1,1))
data =  DataLoader(torch.from_numpy(dataset).float(), batch_size=10, shuffle=True)


print("Entradas:")
print(X)
print("Salidas:")
print(Y)
print("Observe que hay 768 ejemplos y 8 atributos:")
print(len(X))
print("Observe que hay 768 etiquetas")
print((Y.shape))

Entradas:
Salidas:
tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
 

In [228]:
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

class FFNN(Module):

  def __init__(self, d0=8, d1=12):
    super(FFNN, self).__init__()
    
    #self.fc_1 = Linear(d0,d1)
    #self.fc_2 = Linear(d1,d0) 
    #self.fc_out = Linear(d0,1)
    #self.relu = nn.ReLU()
    #self.sigmoid = nn.Sigmoid()
    relu=nn.ReLU
    
    layer_list=[
          ("input",Linear(d0,d1)),
          ('relu1', relu()),
          ('linear', Linear(d1,d0)),
          ('relu2', relu()),
          ('output',Linear(d0,1)),
    #      ('sigmoid', sigmoide()),    
    ]
    layerDict = OrderedDict(layer_list)
    self.layers = Sequential(layerDict)
    self.apply(self.init_weights)
    self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    
  def forward(self, x):
    output = self.layers.input(x)
    output = self.layers.relu1(output)
    output = self.layers.linear(output)
    output = self.layers.relu2(output)
    output = self.layers.output(output)
    output = torch.sigmoid(output)
    return output
  
  def evaluate(self,predict):
    return (predict > 0.5).float()
  
  def accuracy(self,y_test,x_test): 
    y_pred = self.evaluate(x_test)
    acc=accuracy_score(y_test.numpy(), y_pred.numpy())
    print(f'Accuracy: {acc}')
  
  @torch.no_grad()
  def init_weights(self,m):
    if isinstance(m, nn.Linear):
      nn.init.uniform_(m.weight)
      m.bias.fill_(0.0)

    

In [229]:
model=FFNN()
criterion = nn.BCELoss()
model

FFNN(
  (layers): Sequential(
    (input): Linear(in_features=8, out_features=12, bias=True)
    (relu1): ReLU()
    (linear): Linear(in_features=12, out_features=8, bias=True)
    (relu2): ReLU()
    (output): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [ ]:
epochs=1000
model_params=[]
for epoch in range(epochs):
    model.train()
    #iteration_loss,iteration_accuracy = 0.,0.
    #for i, batch_data in enumerate(data):
    #    batch_size = len(batch_data)
    #    input_batch, output_batch = batch_data[:,0:8], batch_data[:,8].reshape(-1,1)
    #    prediction = model(input_batch)
    #    loss = criterion(prediction, output_batch)
    #    optimizer.zero_grad()
    #    loss.backward()
    #    optimizer.step()
    #    iteration_loss += loss.item()
    input, output = inputs,Y
    prediction = model.forward(input)
    pre=model.parameters()
    loss = criterion(prediction, output)
    model.optimizer.zero_grad()
    loss.backward()
    model.optimizer.step()
    pos=model.parameters()
    model_params.append((pos,pre))

    if epoch % 10==0:
        model.eval()
        with torch.no_grad():
            prediction = model(inputs)
            output = model.evaluate(prediction)
            accuracy = accuracy_score(Y.numpy(), output.numpy())
        print(f'Epoch {epoch+1}/{epochs} loss: {loss.item()} Accuracy {accuracy:.5f}')
        
confusion_matrix(Y.numpy(), output.numpy())

In [266]:

#otra forma de defnir el modelo
relu=nn.ReLU
model1 = Sequential(OrderedDict([
          ("input", Linear(8, 12)),
          ('relu1', relu()),
          ('linear', nn.Linear(12,8)),
          ('relu2', relu()),
          ('output', nn.Linear(8, 1)),
        ]))

@torch.no_grad()
def init_weights(m):
  if isinstance(m, nn.Linear):
    nn.init.uniform_(m.weight)
    m.bias.fill_(0.0)
model1.apply(init_weights)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model1.parameters())
m = nn.Sigmoid()

epochs=10000

for epoch in range(epochs):
    model1.train()
    input, output = inputs,Y
    optimizer.zero_grad()
    prediction = model1(input)
    loss = criterion(m(prediction), output)

    loss.backward()
    optimizer.step()
    
    correct = 0
    total = 0

    if epoch % 10==0:
        model.eval()
        with torch.no_grad():
            output = m(model(inputs))
            predicted = (output > 0.5).float()
            total += Y.shape[0]
            correct += int((predicted == Y).sum())
        print(f'Epoch {epoch+1}/{epochs} loss: {loss.item()} Accuracy { correct / total:.5f}')
        


Epoch 1/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 11/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 21/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 31/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 41/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 51/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 61/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 71/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 81/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 91/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 101/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 111/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 121/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 131/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 141/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 151/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 161/10000 loss: 65.10416412353516 Accuracy 0.34896
Epoch 171/10000 loss: 65.10416412353516 Ac